# Milestone 1
Group 4: Mengjun Chen, Qurat-ul-Ain Azim, Ashwin Babu, Mohammad Reza Nabizadeh
## 1. Team-work contract
- Link to the team work contract:
https://docs.google.com/document/d/1FKmLnss40QeTBvXecWfMuavG4hm0lxKGkV9BtIFVIS8/edit
## 2. Creating a repository and project structure
- Link to the repository
https://github.com/UBC-MDS/525_group_4

## 3. Downloading the data

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
%cd ~/Desktop/MDS/525/525_group_4

/Users/mrnabiz/Desktop/MDS/525/525_group_4


In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 1.62 s, sys: 1.82 s, total: 3.44 s
Wall time: 43.2 s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 12.7 s, sys: 1.61 s, total: 14.3 s
Wall time: 15.2 s


In [8]:
%ls -ltr figshare/

total 12086184
-rw-r--r--   1 mrnabiz  staff  814041183 Apr  1 09:02 data.zip
-rw-r--r--   1 mrnabiz  staff   95376895 Apr  1 09:02 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff   94960113 Apr  1 09:02 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff   82474546 Apr  1 09:02 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  127613760 Apr  1 09:02 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  232118894 Apr  1 09:02 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  330360682 Apr  1 09:02 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  254009247 Apr  1 09:02 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  235661418 Apr  1 09:02 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  294260911 Apr  1 09:02 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  staff  295768615 Apr  1 09:02 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 mrnabiz  s

## 4. Combining data CSVs

In [9]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
files = glob.glob('figshare/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figshare/combined_data.csv")

CPU times: user 5min 10s, sys: 10.8 s, total: 5min 21s
Wall time: 5min 24s


### Results Comparison:
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mengjun Chen    | MacOS    |  8GB   |    Apple M1       |   Yes     |   17 Mins         |
| Qurat-ul-Ain Azim    |     MacOS             |   8GB  |     Intel      |      No   | 22 Mins and 14 Seconds |
| Ashwin Babu    |      MacOS            |  8GB   |    Apple M2       |   Yes    |   18 Mins         |
| Mohammad Reza Nabizadeh | MacOS | 16GB | Apple M2 | YES | 5 Mins and 24 Seconds|

## 5. Load the combined CSV to memory and perform a simple EDA
### Changing `dtype`of data

In [10]:
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)", "model"]
df = pd.read_csv("figshare/combined_data.csv",usecols=use_cols)

In [11]:
%%time
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 3.13 s, sys: 149 ms, total: 3.28 s
Wall time: 3.27 s


In [12]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB
CPU times: user 3.1 ms, sys: 2.18 ms, total: 5.28 ms
Wall time: 7.45 ms


In [13]:
%%time
df.describe()

CPU times: user 11.3 s, sys: 3.06 s, total: 14.3 s
Wall time: 14.6 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [14]:
print(f"Memory usage with float64: {df[['lat_min', 'lat_max', 'lon_min' ,'lon_max', 'rain (mm/day)', 'model']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min', 'lat_max', 'lon_min' ,'lon_max', 'rain (mm/day)', 'model']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2998.46 MB
Memory usage with float32: 1749.10 MB


In [15]:
df[['lat_min', 'lat_max', 'lon_min' ,'lon_max', 'rain (mm/day)', 'model']] = \
df[['lat_min', 'lat_max', 'lon_min' ,'lon_max', 'rain (mm/day)', 'model']].astype('float32', errors='ignore')

In [16]:
%%time
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 3.11 s, sys: 39.2 ms, total: 3.15 s
Wall time: 3.14 s


In [17]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float32
 2   lat_max        float32
 3   lon_min        float32
 4   lon_max        float32
 5   rain (mm/day)  float32
 6   model          object 
dtypes: float32(5), object(2)
memory usage: 2.1+ GB
CPU times: user 2.87 ms, sys: 665 µs, total: 3.54 ms
Wall time: 3.37 ms


In [18]:
%%time
df.describe()

CPU times: user 10.5 s, sys: 1.29 s, total: 11.8 s
Wall time: 11.8 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310497e+01,-3.197765e+01,1.469057e+02,1.482150e+02,1.901173e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


### Results Comparison:
- **Mengjun Chen:**
I chunk the entire dataset into chunks, I picked the first one:
  - The `corr` time is 14 ms for 20000 sample chunk. 
  - The `info` time is 44.1 ms.
  - The `describe` time is 30.8 ms.
- **Qurat-ul-Ain Azim:**
By changing the type the float values from float 64 to float 32:
  - The `value_counts` time changed from 18.2 s to 16.7 s.
  - The `info` time changed from 55.3 ms to 59.1 ms.
  - The `describe` time changed from 49.3 s to  40.4 s.

- **Ashwin Babu:**
By changing the type the float values from float 64 to float 32:
  - The `value_counts` time changed from 16.2 s to 13.7 s.
  - The `info` time changed from 47.3 ms to 53.1 ms.
  - The `describe` time changed from 45.3 s to  37.2 s.

- **Mohammad Reza Nabizadeh:**
By changing the type the float values from float 64 to float 32:
  - The `value_counts` time dropped from 3.14 s to 3.09 s.
  - The `info` time dropped from 4.35 ms to 6.81 ms.
  - The `describe` time dropped from 15 s to 11.8 s.
  
### Load just columns that we want

In [19]:
use_cols = ["time", "lat_min", "lon_min", "rain (mm/day)", "model"]
df = pd.read_csv("figshare/combined_data.csv",usecols=use_cols)

In [20]:
%%time
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 3.11 s, sys: 106 ms, total: 3.21 s
Wall time: 3.21 s


In [21]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lon_min        float64
 3   rain (mm/day)  float64
 4   model          object 
dtypes: float64(3), object(2)
memory usage: 2.3+ GB
CPU times: user 2.86 ms, sys: 1.36 ms, total: 4.22 ms
Wall time: 3.96 ms


In [22]:
%%time
df.describe()

CPU times: user 7.42 s, sys: 2.05 s, total: 9.47 s
Wall time: 9.63 s


,lat_min,lon_min,rain (mm/day)
count,5.924854e+07,5.924854e+07,5.924854e+07
mean,-3.310482e+01,1.469059e+02,1.901170e+00
std,1.963549e+00,3.793784e+00,5.585735e+00
min,-3.646739e+01,1.406250e+02,-3.807373e-12
25%,-3.486911e+01,1.434375e+02,3.838413e-06
50%,-3.300000e+01,1.468750e+02,6.154947e-02
75%,-3.140170e+01,1.501875e+02,1.020918e+00
max,-2.990000e+01,1.537500e+02,4.329395e+02


### Results Comparison:
- **Mengjun Chen:**
By just selecting the columns of the data we need with in one single chunk:
  - The `info` time is 40.3 ms.
  - The `describe` time is 32.3 ms.

- **Qurat-ul-Ain Azim:**
By just selecting the columns of the data we need:
  - The `value_counts` time changed from 18.2 s to 15.9 s.
  - The `info` time changed from 55.3 ms to 78.9 ms.
  - The `describe` time changed from 49.3 s to  38.5 s.
  
- **Ashwin Babu:**
By just selecting the columns that we want from the data:
 - The `info` time changed from 53ms to 65ms.
 - The `value_counts` time changed from 16 to 11 s.
 - The `describe` time changed from 43 s to 37 s.  

- **Mohammad Reza Nabizadeh:**
By just selecting the columns of the data we need:
  - The `value_counts` time dropped from 3.14 s to 3.09 s.
  - The `info` time dropped from 4.35 ms to 5.78 ms.
  - The `describe` time dropped from 15 s to 8.85 s.

### Performing EDA in R

In [24]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [25]:
%%time
%%R -i df
start_time <- Sys.time()
suppressMessages(library(dplyr))
result <- df |> count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

              model       n
1        ACCESS-CM2 1932840
2     ACCESS-ESM1-5 1610700
3    AWI-ESM-1-1-LR  966420
4       BCC-CSM2-MR 3035340
5          BCC-ESM1  551880
6      CMCC-CM2-HR4 3541230
7      CMCC-CM2-SR5 3541230
8         CMCC-ESM2 3541230
9           CanESM5  551880
10 EC-Earth3-Veg-LR 3037320
11      FGOALS-f3-L 3219300
12        FGOALS-g3 1287720
13         GFDL-CM4 3219300
14        GFDL-ESM4 3219300
15        INM-CM4-8 1609650
16        INM-CM5-0 1609650
17        KIOST-ESM 1287720
18           MIROC6 2070900
19  MPI-ESM-1-2-HAM  966420
20    MPI-ESM1-2-HR 5154240
21    MPI-ESM1-2-LR  966420
22       MRI-ESM2-0 3037320
23            NESM3  966420
24       NorESM2-LM  919800
25       NorESM2-MM 3541230
26      SAM0-UNICON 3541153
27          TaiESM1 3541230
Time difference of 3.338877 secs
CPU times: user 11min 58s, sys: 1min 32s, total: 13min 31s
Wall time: 14min 5s
